In [11]:
import math

In [12]:
m_bch = 14.8 # кг - масса боевой части
m_dop = 1.5 # кг - суммарная масса других элементов, входящих в отсек БЧ
rho_osk = 9800 # кг/м^3 - плотность ГПЭ

a, b, c = 20e-3, 10e-3, 50e-3 # м - грани параллепипеда ГПЭ
N = 80

m_gpe = N*a*b*c*rho_osk
m_gpe_1 = a*b*c*rho_osk
m_vv = m_bch - m_gpe - m_dop # кг - масса ВВ

D = 8.3 # км/с - скорость детонации ВВ

In [9]:
print(round(m_gpe, 5), m_gpe_1, m_vv)

7.84 0.098 5.46


Для ГПЭ теоретическая формула для определения их начальной скорости воспользуемся формулой для плоских тел

In [10]:
beta = m_vv / m_gpe
print(beta)

0.6964285714285714


In [14]:
def get_V_ob(D, beta):
    return 1.2*D*(math.sqrt(1+32/27*beta) - 1)/(math.sqrt(1+32/27*beta) + 1)

def get_F(S_m, a, b, c):
    return S_m / (a*b*c)**(2/3)

V_ob = get_V_ob(D, beta)
print(V_ob)

1.4872709647507776


In [16]:
class obj:
    def __init__(self, b_pregr, rho_p1, H_01):
        self.b_pregr = b_pregr
        self.rho_p1 = rho_p1
        self.H_01 = H_01
        self.a, self.b, self.c = 20e-3, 10e-3, 50e-3 # м - грани параллепипеда ГПЭ
        
        self.S_m = None
        self.F = None
        
        
class Bairactar(obj):
    def __init__(self, b_pregr, rho_p1, H_01):
        super().__init__(b_pregr, rho_p1, H_01)
    
    def calculate(self):
        self.S_m = (a*b+a*c)/2
        self.F = get_F(self.F, self.a, self.b, self.c)

    

class FF_18A(obj):
    def __init__(self, b_pregr, rho_p1, H_01):
        super().__init__(b_pregr, rho_p1, H_01)

In [ ]:
obj_1 = Bairactar(b_pregr=3, pho_p1=1000, H_01=100)